<a href="https://colab.research.google.com/github/Dillonreed/3831-Coursework-Part-2/blob/main/3831_Coursework_Part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Starter point for the Machine Learning element of the coursework

This part of the coursework uses the same dataset as used in the first part of the coursework. In this part of the coursework you are going to develop three machine learning models for predicting the **'median_house_value'** from the dataset. The focus here is on comparing the different models and looking at how you can improve them. You do not need to use a cleaned up version of the data (i.e. perform outlier removal in advance) but you may if you wish although please comment on this.

Everything needed to complete this assignment should be available in the course slides, but external information from the interenet may prove useful and is encouraged. Please provide citations for resources used in the course.

### Load libraries and read in data

In [ ]:
import pandas as pd
houses = pd.read_csv('https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/IMPUTATION/TARGET-DATASETS/ORIGINAL/houses.csv', header=0)

### (Optional) Preprocessing and Data Exploration
What data exploration would you perform to get a sense of the data, and what preprocessing outside of feature normalasation would you perform? This section is an optional place for students to do additional work on the data before progressing into the assignment proper.

## Data Preparation
In this section you will perform feature selection, feature normalisation, and provide a rationale for your actions.

### Feature Selection
Think about which features may be useful in predicting **'median_house_value'**, are all features in the provided data set useful? Is a subset all that's needed? What techniques can you utilise to make this determination?

### Feature Normalisation
Think about what normalisation/standardisation methods you should apply to the dataset given what you understand about the raw data.

### Rationale
Provide your rationale for both Feature Selection and Feature Normalisation here.

## Train, Validate, Test Split
In this section you will perform a train, validate, test split utilisation the knowledge learned in class and provide a rationale for your actions.

### Perform Data Split
Utilise the information from class or online to split your data into train, validate, and test partitions.

### Rationale
Provide a rationale for how you made your train, test, split decision.

## Metric Selection
In this section your will make appropriate metric selection for analysing your models and provide a rationale.

### Select Appropriate Metrics
Think about the models you are building, and what the appropriate metrics and scoring should be for those models.

### Rationale
Provide a rationale for your metrics and scoring selection.

## Machine Learning Model 1
In this section your will select an appropriate machine learning model for predicting **'median_house_value'**, apply it to the dataset to perform this prediction on the test set created in the Train, Validate, Test Split section, and comment on the predictive ability of the model you selected.

### Select and Build a Machine Learning Model 1
Think about the task at hand, and select an appropriate model to build on the train and validate data. Try different sets of hyper-parameters to improve your model.

### Use Model 1 to Predict on Test Data
Use the model you've trained to predict **'median_house_value'** on the test data.

### Rationale
Provide a rationale for model

### Comment on Predictive Ability
Think about the metrics and scoring received from the training and testing components. Think about the generalisability and quality of your results.

## Machine Learning Model 2
In this section your will select an appropriate machine learning model for predicting **'median_house_value'**, apply it to the dataset to perform this prediction on the test set created in the Train, Validate, Test Split section, and comment on the predictive ability of the model you selected.

### Select and Build a Machine Learning Model 2
Think about the task at hand, and select an appropriate model to build on the train and validate data. Try different sets of hyper-parameters to improve your model.

### Use Model 2 to Predict on Test Data
Use the model you've trained to predict **'median_house_value'** on the test data.

### Rationale
Provide a rationale for model

### Comment on Predictive Ability
Think about the metrics and scoring received from the training and testing components. Think about the generalisability and quality of your results.

## Machine Learning Model 3
In this section your will select an appropriate machine learning model for predicting **'median_house_value'**, apply it to the dataset to perform this prediction on the test set created in the Train, Validate, Test Split section, and comment on the predictive ability of the model you selected.

### Select and Build a Machine Learning Model 3
Think about the task at hand, and select an appropriate model to build on the train and validate data. Try different sets of hyper-parameters to improve your model.

### Use Model 3 to Predict on Test Data
Use the model you've trained to predict **'median_house_value'** on the test data.

### Rationale
Provide a rationale for model

### Comment on Predictive Ability
Think about the metrics and scoring received from the training and testing components. Think about the generalisability and quality of your results.

## Comparison Between Models
In this section you will comment on the difference in predictive ability between models, the difference in analysis metrics between models, and the pros/cons of each model as you understand it.

### Difference in Predictive Ability
Think about what you understand if the predictive ability from each model based on scores gained, comment on the difference between each model and suggest why you believe this to be the case. Additionally comment on the difference in hyper-parameter selection between your models.

### Difference in Analysis Metrics
Think about the analysis metrics you received from each model, comment on the use in the difference of these metrics and suggesting which model is "better" or "worse" when comparing them.

### Pros/Cons
Think about the models and analysis metrics you've gathered, and suggest the positives and negatives of each model.

## Conclusion
In this section you will draw conclusions about the models you've selected, and efficacy of different analysis metrics you've used, and any additional comments or concerns about the models or metrics you've observed.

### Model Selection
Think about all of the information you've learned both theoretically from the slides and practically through completing this assingment about the models you've selected. What conclusions can you draw about the models, and what situations would you pick one model over another?

### Analysis Metrics
Think about the metrics you've selected and what you've learned both thetoretically from the slides and practically through this assignment. What conclusions can you draw about these metrics, and what situations might you pick one set of metrics over another? (Here you may comment on metrics which you have not selected in this assignment)

### Any Additional Conclusions
For those of you which have noticed things or made conclusions which do not neatly fit into any other section, please put them here.